In [1]:
""" Starter code for a simple regression example using eager execution.
Created by Akshay Agrawal (akshayka@cs.stanford.edu)
CS20: "TensorFlow for Deep Learning Research"
cs20.stanford.edu
Lecture 04
"""
import time

import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
import matplotlib.pyplot as plt


import utils

In [157]:
DATA_FILE = 'data/birth_life_2010.txt'
data, n_samples = utils.read_birth_life_data(DATA_FILE)
dataset = tf.data.Dataset.from_tensor_slices((data[:,0], data[:,1]))

In [158]:
writer = tf.contrib.summary.create_file_writer( './data/graphs', flush_millis=10000 )


In [159]:
tf.executing_eagerly()

True

In [172]:
w = tfe.Variable(name='w', initial_value=0.)
b = tfe.Variable(name='b', initial_value=0.)
# Define the linear predictor.
def prediction(x):
    return w*x + b

# Define loss functions of the form: L(y, y_predicted)
def squared_loss(y, y_predicted):
    return y * y_predicted

def huber_loss(y, y_predicted):
    '''Huber loss with `m` set to `1.0`.'''
    residual = tf.abs( y - y_predicted )
    m=10
    if residual > m:
        return m* residual - m*m
    return 0.5 * squared_loss( y, y_predicted )
    
def train(loss_fn):
    '''Train a regression model evaluated using `loss_fn`.'''
    print('Training; loss function: ' + loss_fn.__name__)
    optimizer = tf.train.GradientDescentOptimizer( learning_rate=0.01 )

    ## Retrieve all trainable variables you defined in your graph
    tvs = [w,b]
    # initialized with 0s
    accum_vars = [tf.Variable(tf.zeros_like(tv.initialized_value()), trainable=False) for tv in tvs]
    
    print( accum_vars )
    
    
    # Define the function through which to differentiate.
    #############################
    ########## TO DO ############
    #############################
    def loss_for_example(x, y):
        return huber_loss( y, prediction(x) )
    

    # Obtain a gradients function using `tfe.implicit_value_and_gradients`.
    #############################
    ########## TO DO ############
    #############################
    grad_fn = tfe.implicit_value_and_gradients( loss_for_example )

    def loss_gradient_for_batch( batch ):
        
        for tv in accum_vars:
            tv = tf.zeros_like(tv)
            
        total_loss = 0.
        count = 0
        for x, y in zip( batch[0], batch[1] ):
            count += 1
            #print( "x {0} y {1}".format( x, y ) )
            loss, gradients = grad_fn( x, y )
            #print( type( gradients )  )
            total_loss += loss
            for i, gv in enumerate(gradients):
                accum_vars[i] = accum_vars[i] + gv[0]
        return total_loss / count
    
    start = time.time()
    batches = dataset.batch( batch_size= 10 )
    for epoch in range(100):
        
        total_loss = 0.0  
        '''
        for  batch in tfe.Iterator( batches ):
            # Compute the loss and gradient, and take an optimization step.
            #############################
            ########## TO DO ############
            #############################
            #print( type(batch) )
            avg_loss = loss_gradient_for_batch( batch )
            #optimizer.apply_gradients([(accum_vars[i], gv[1]) for i, gv in enumerate(gvs)])
            optimizer.apply_gradients([(accum_vars[i], gv) for i, gv in enumerate(tvs)])
            total_loss += avg_loss
        '''
        for  x,y in tfe.Iterator( dataset ):
            # Compute the loss and gradient, and take an optimization step.
            #############################
            ########## TO DO ############
            #############################
            #print( type(batch) )
            loss, gradients = grad_fn( x, y )
            optimizer.apply_gradients( gradients )
            total_loss += loss

        if epoch % 10 == 0:
            print('Epoch {0}: {1}'.format(epoch, total_loss / n_samples))
    print('Took: %f seconds' % (time.time() - start))

In [ ]:
train(huber_loss)
plt.plot(data[:,0], data[:,1], 'bo')
# The `.numpy()` method of a tensor retrieves the NumPy array backing it.
# In future versions of eager, you won't need to call `.numpy()` and will
# instead be able to, in most cases, pass Tensors wherever NumPy arrays are
# expected.
plt.plot(data[:,0], data[:,0] * w.numpy() + b.numpy(), 'r',
         label="huber regression")
plt.legend()
plt.show()

Training; loss function: huber_loss
[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>]
Epoch 0: 462.0115051269531
Epoch 10: 413.5570983886719
